# **Problem 3: Feature Selection and Classification Using Weighted K-Nearest Neighbors (KNN)**

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

**1. Data Preparation:**


In [3]:
df = pd.read_csv('P3.csv')

In [4]:
df.head(5)  # or .tail()

,Unnamed: 0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,label
0,0,0.140023,0.265418,-0.535880,0.808262,-0.998551,1.147053,0.979441,0.722351,0.559936,0.399382,1.098400,0.036805,1.302542,-0.239813,0.625914,0.0
1,1,1.176099,1.690023,-0.737042,1.207350,-1.249740,-0.921881,0.065195,-0.581772,0.645254,-0.089174,-1.571598,0.633757,0.636572,1.233946,-0.300362,1.0
2,2,1.322751,0.635943,0.012525,0.027479,0.083267,0.447982,1.158902,1.177922,0.887521,0.018378,-0.395321,0.451929,0.744254,-0.492224,-0.316476,1.0
3,3,0.907289,-0.475599,-1.270970,1.837189,-2.471438,1.270536,-0.536260,-0.852590,0.778421,0.362119,-1.111151,0.153531,-0.342588,-0.147104,-0.585484,0.0
4,4,0.809604,1.039083,0.869025,-1.204626,1.756469,0.318814,-2.582921,-1.493409,-1.221354,0.435062,-0.068407,0.942154,0.329038,0.375319,-0.843292,1.0


In [5]:
# Normalization of numerical features
features = df.iloc[:,:-1]
target_value = df.iloc[:,-1]

def minmax_norm(X:pd.Series):
    return (X - np.min(X)) / (np.max(X) - np.min(X))

for col in features.columns:
    df[col] = minmax_norm(df[col])

In [6]:

def stratified_train_test_split(df, target_column, test_size=0.3):
    train_set = pd.DataFrame()
    test_set = pd.DataFrame()

    classes = df[target_column].unique()

    for class_ in classes:
        class_subset = df[df[target_column] == class_]
        
        test_subset_size = int(len(class_subset) * test_size)
        
        test_subset = class_subset.sample(test_subset_size)
        train_subset = class_subset.drop(test_subset.index)

        train_set = pd.concat([train_set, train_subset])
        test_set = pd.concat([test_set, test_subset])

    return train_set, test_set


In [7]:
train_df , test_df = stratified_train_test_split(df,'label',0.1)

In [8]:
train_df.shape

(270, 17)

In [9]:
test_df.shape

(30, 17)

**2. Weighted KNN Loss Calculation:**

In [10]:
def weighted_knn(weights, train_points:pd.DataFrame, new_points:pd.DataFrame ,k = 5 ,target_value_name = 'label'):
    predictions = []
    for _, point in new_points.iterrows():
        distances = []
        for _, p in train_points.iterrows():
            counter = 0
            distance = 0
            for col in train_points.columns:
                if col != 'label' and col != 'Unnamed: 0':
                    distance += (weights[counter] * ((point[col] - p[col])**2))
                    counter += 1
            distance = np.sqrt(distance)
            distance = round(distance,ndigits=2)
            distances.append((distance,p[target_value_name]))
        # print(distances)
        distances = sorted(distances)[:k]

        counter_class_0 = 0
        counter_class_1 = 0

        for d in distances:
            if d[1] == 0:
                counter_class_0 += 1
            else:
                counter_class_1 += 1

        if counter_class_0 > counter_class_1:
            predictions.append(0)
        else:
            predictions.append(1)
        
    return predictions

In [11]:
weights = [1 for i in range(15)]
pred = weighted_knn(weights ,train_df,test_df.iloc[:,:-1],k=5,target_value_name='label')

In [12]:
pred

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1]

In [13]:
def confusion_matrix(Y_true,Y_pred):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in range(len(Y_pred)):
        if Y_pred[i] == Y_true[i]:
            if Y_true[i] == 0:
                TN += 1
            elif Y_true[i] == 1:
                TP += 1
        if Y_pred[i] != Y_true[i]:
            if Y_true[i] == 0:
                FP += 1
            elif Y_true[i] == 1:
                FN += 1
    return TP,TN,FP,FN

def precision(Y_true,Y_pred):
    TP,TN,FP,FN = confusion_matrix(Y_true,Y_pred)
    if TP+FP == 0:
        return np.inf
    return (TP)/(TP+FP)

def recall(Y_true,Y_pred):
    TP,TN,FP,FN = confusion_matrix(Y_true,Y_pred)
    return (TP) / (TP + FN)

def f1_score(Y_true,Y_pred):
    TP,TN,FP,FN = confusion_matrix(Y_true,Y_pred)
    precision_ = precision(Y_true,Y_pred)
    recall_ = recall(Y_true,Y_pred)
    return (2*precision_*recall_)/(precision_ + recall_)


In [14]:
print(f'f1 score for weighted knn: {f1_score(np.array(test_df.iloc[:,-1]),np.array(pred))}')

f1 score for weighted knn: 0.8148148148148148


In [15]:
def log_loss(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [16]:
print(f'log loss for weighted knn: {log_loss(np.array(test_df.iloc[:,-1]),np.array(pred))}')

log loss for weighted knn: 5.756489385732788


In [17]:
def log_loss_for_weighted_knn(weights ,train_df,test_df,k=5,target_value_name='label'):
    pred = weighted_knn(weights ,train_df,test_df.iloc[:,:-1],k=5,target_value_name='label')
    print(f'log loss for weighted knn: {log_loss(np.array(test_df.iloc[:,-1]),np.array(pred))}')
    return log_loss(np.array(test_df.iloc[:,-1]),np.array(pred))

In [18]:
log_loss_for_weighted_knn(weights ,train_df,test_df,k=5,target_value_name='label')

log loss for weighted knn: 5.756489385732788


np.float64(5.756489385732788)

**3. Weight Optimization:**

In [19]:
from scipy.optimize import minimize

initial_weights = np.zeros(train_df.shape[1] - 2)
result = minimize(log_loss_for_weighted_knn, initial_weights, args=(train_df, test_df, 5, 'label'), 
                  method='L-BFGS-B', bounds=[(0, 2)],
                  options={'disp': True, 'eps': 1e-2})
disp=True

log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 12.664218011467248
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 14.966803104461292
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 17.269388197455335
log loss for weighted knn: 8.059127785222179
log loss for weighted knn: 6.907808585477483
log loss for weighted knn: 8.059127785222179
log loss for weighted knn: 8.059127785222179
log loss for weighted knn: 8.059127785222179
log loss for weighted knn: 8.0591277852

In [20]:
optimized_weights = result.x

In [21]:
optimized_weights

array([0.31578986, 2.        , 0.        , 0.        , 0.        ,
       0.31578986, 0.63157972, 2.        , 0.        , 0.        ,
       0.        , 0.        , 0.31578986, 0.31578986, 0.31578986])

**4. Weighted and Vanilla KNN Classification:**

In [22]:
normal_knn_error = log_loss_for_weighted_knn([1 for i in range(15)],train_df,test_df)
print(f'normal knn result error : {normal_knn_error}')

weighted_knn_error = log_loss_for_weighted_knn(optimized_weights,train_df,test_df)
print(f'weighted knn result error : {weighted_knn_error}')

log loss for weighted knn: 5.756489385732788
normal knn result error : 5.756489385732788
log loss for weighted knn: 5.756489385732788
weighted knn result error : 5.756489385732788


**5. Dimensionality Reduction Using Random Feature Subsets:**

In [23]:
import random
import traceback

def select_5_subset_features():
    feature_subset = []
    feature_indices = []
    for i in range(5):
        random_ = random.randint(1,15)
        feature_subset.append(f'f{random_}')
        feature_indices.append(random_)
    return feature_subset,feature_indices

best_feature_subsets = []
best_feature_indices = []
best_error = 20
i = 0
while i < 8:
    try:
        feature_s ,feature_i = select_5_subset_features()
        print(feature_s)
        feature_s.append('label')
        tr_df = train_df[feature_s]
        te_df = test_df[feature_s]
        #  weights =  optimized_weights[feature_i]
        weights = [1,1,1,1,1]
        error = log_loss_for_weighted_knn(weights,tr_df,te_df)
        if error <  best_error:
            best_error = error
            best_feature_subsets = feature_s
            best_feature_indices = feature_i
        i += 1
    except Exception as e:
        print(f'Reselecting')

print(f'best error : {best_error} \nbest features subset : {best_feature_subsets}')

['f8', 'f13', 'f15', 'f5', 'f13']
Reselecting
['f6', 'f4', 'f3', 'f13', 'f12']
log loss for weighted knn: 12.66432462445794
['f3', 'f10', 'f6', 'f9', 'f6']
Reselecting
['f10', 'f8', 'f9', 'f11', 'f7']
log loss for weighted knn: 13.815643824202636
['f3', 'f3', 'f2', 'f11', 'f1']
Reselecting
['f5', 'f11', 'f8', 'f15', 'f12']
log loss for weighted knn: 10.361739531463895
['f9', 'f12', 'f1', 'f10', 'f8']
log loss for weighted knn: 20.723532369423136
['f12', 'f10', 'f12', 'f5', 'f4']
Reselecting
['f13', 'f12', 'f10', 'f14', 'f11']
log loss for weighted knn: 17.269548116941376
['f3', 'f6', 'f2', 'f1', 'f1']
Reselecting
['f10', 'f14', 'f14', 'f10', 'f15']
Reselecting
['f9', 'f12', 'f2', 'f14', 'f5']
log loss for weighted knn: 8.059074478726833
['f5', 'f8', 'f15', 'f2', 'f13']
log loss for weighted knn: 8.059101131974506
['f5', 'f8', 'f5', 'f8', 'f7']
Reselecting
['f1', 'f5', 'f12', 'f3', 'f10']
log loss for weighted knn: 11.513085384456266
best error : 8.059074478726833 
best features subset 

**6. Feature Selection Based on Weights:**

In [24]:
indices_of_top_5 = np.argsort(optimized_weights)[-5:]

In [25]:
indices_of_top_5

array([13, 14,  6,  1,  7])

In [26]:
optimized_weights[indices_of_top_5]

array([0.31578986, 0.31578986, 0.63157972, 2.        , 2.        ])

In [27]:
selected_features = []
for i  in range(len(indices_of_top_5)):
    index = indices_of_top_5[i] + 1
    selected_features.append(f'f{index}')
print(selected_features)
columns = selected_features
columns.append('label')
tr_df = train_df[columns]
te_df = test_df[columns]
error = log_loss_for_weighted_knn(optimized_weights[indices_of_top_5],tr_df,te_df)
print(f'error of the selected 5 features : {error}')

['f14', 'f15', 'f7', 'f2', 'f8']
log loss for weighted knn: 9.210393678471528
error of the selected 5 features : 9.210393678471528


**7. Comparison and Analysis:**

In [28]:
# no feature selection
print(f'log loss for no feature selection : {log_loss_for_weighted_knn(weights=[1 for i in range(15)],train_df=train_df,test_df=test_df)}')
# for random feature selection 
tr_df = train_df[best_feature_subsets]
te_df = test_df[best_feature_subsets]
print(f'log loss for random feature selection : {log_loss_for_weighted_knn(weights=[1 for i in range(5)],train_df=tr_df,test_df=te_df)}')
# for weighted feature selection 
tr_df = train_df[columns]
te_df = test_df[columns]
print(f'log loss for weighted feature selection : {log_loss_for_weighted_knn(weights=optimized_weights[indices_of_top_5],train_df=tr_df,test_df=te_df)}')


log loss for weighted knn: 5.756489385732788
log loss for no feature selection : 5.756489385732788
log loss for weighted knn: 8.059074478726833
log loss for random feature selection : 8.059074478726833
log loss for weighted knn: 9.210393678471528
log loss for weighted feature selection : 9.210393678471528
